In [2]:
# Setup and imports for all examples
import os
import sys
import asyncio
from pydantic import BaseModel, Field
from typing import List, Optional
import json

from llmservice.generation_engine import GenerationEngine
from llmservice.schemas import GenerationRequest
from llmservice import BaseLLMService

print("Imports successful! Ready to run examples.")

Imports successful! Ready to run examples.


# LLMService Examples - Migrated to Structured Outputs

This notebook demonstrates the three examples from the `examples/` directory, now using structured outputs instead of the removed pipeline system:

1. **Capital Finder** - Extract capital cities
2. **SQL Code Generator** - Generate SQL queries  
3. **Translator** - Translate text to different languages

All examples now use Pydantic schemas for guaranteed structured outputs with no parsing errors!

## Example 1: Capital Finder

Previously used SemanticIsolation pipeline to extract just the capital. Now uses structured output with a Pydantic schema.

In [3]:
# Define schema for capital extraction
class CapitalInfo(BaseModel):
    country: str = Field(description="The country name")
    capital: str = Field(description="The capital city")
    confidence: Optional[float] = Field(default=None, ge=0, le=1, description="Confidence score")

class CapitalFinderService(BaseLLMService):
    
    def ask_llm_to_tell_capital(self, user_input: str):
        """Ask LLM about capital (unstructured response)"""
        prompt = f"What is the capital of {user_input}? Provide a detailed answer."
        
        generation_request = GenerationRequest(
            user_prompt=prompt,
            model="gpt-4o-mini"
        )
        
        return self.execute_generation(generation_request)
    
    def bring_only_capital(self, user_input: str):
        """Extract just the capital using structured output (replaces SemanticIsolation)"""
        prompt = f"What is the capital of {user_input}?"
        
        # Use structured output instead of pipeline
        generation_request = GenerationRequest(
            user_prompt=prompt,
            system_prompt="Extract the country and its capital city",
            response_schema=CapitalInfo,  # ← Structured output!
            model="gpt-4o-mini"
        )
        
        result = self.execute_generation(generation_request)
        
        if result.success:
            # Parse the structured response
            data = json.loads(result.content)
            capital_info = CapitalInfo(**data)
            # Return just the capital for compatibility
            result.content = capital_info.capital
        
        return result

# Test Capital Finder
capital_service = CapitalFinderService()
country = "Turkey"

print("=" * 50)
print("Full Response:")
result1 = capital_service.ask_llm_to_tell_capital(country)
print(result1.content)

print("\n" + "=" * 50)
print("Just the Capital (Structured):")
result2 = capital_service.bring_only_capital(country)
print(f"Capital: {result2.content}")

Full Response:
The capital of Turkey is Ankara. 

### Historical Background:
- **Founded**: Although the area has been inhabited for thousands of years, Ankara became prominent as the capital of Turkey in 1923.
- **Strategic Location**: Its central location in Anatolia makes it a vital hub for trade and transportation, connecting various regions of Turkey.

### Government and Political Importance:
- **Capital Status**: Ankara was chosen as the capital primarily for its strategic and symbolic significance to the newly established Republic of Turkey under Mustafa Kemal Atatürk. Atatürk aimed to promote a sense of nationalism and modernity, and moving the capital from Istanbul (then Constantinople) to Ankara was part of this vision.
- **Government Institutions**: The city houses the Turkish Grand National Assembly, the Presidential Palace, and various governmental ministries, making it the political heart of the nation.

### Cultural and Economic Aspects:
- **Cultural Institutions**: Anka

## Example 2: SQL Code Generator

Previously used SemanticIsolation pipeline to extract SQL code. Now uses structured output with SQLQuery schema.

In [4]:
# Define schema for SQL code extraction
class SQLQuery(BaseModel):
    query: str = Field(description="The SQL query")
    explanation: Optional[str] = Field(default=None, description="Brief explanation of what the query does")
    tables_used: List[str] = Field(default_factory=list, description="Tables referenced in the query")

class SQLGeneratorService(BaseLLMService):
    
    def create_sql_code(self, user_question: str, database_desc: str):
        """Generate SQL code using structured output (replaces SemanticIsolation)"""
        
        formatted_prompt = f"""Database description: {database_desc}
        
        User request: {user_question}
        
        Generate the SQL query to fulfill this request."""
        
        # Use structured output instead of SemanticIsolation pipeline
        generation_request = GenerationRequest(
            user_prompt=formatted_prompt,
            system_prompt="Generate SQL query based on the database schema and user request",
            response_schema=SQLQuery,  # ← Structured output!
            model="gpt-4o-mini"
        )
        
        result = self.execute_generation(generation_request)
        
        if result.success:
            # Parse and extract just the SQL query
            data = json.loads(result.content)
            sql_query = SQLQuery(**data)
            # Set content to just the SQL for compatibility
            result.content = sql_query.query
            # Store full info in a custom field if needed
            result.sql_info = sql_query
        
        return result

# Test SQL Generator
sql_service = SQLGeneratorService()

db_desc = """I have a database table with the following schema:
Table: bills
- bill_id (INT, Primary Key)
- bill_date (DATE)
- total (DECIMAL)"""

user_question = "Retrieve the total spendings for each month in the year 2023, grouped by month and ordered chronologically."

result = sql_service.create_sql_code(user_question, db_desc)

if result.success:
    print("=" * 50)
    print("Generated SQL Query:")
    print(result.content)
    
    if hasattr(result, 'sql_info'):
        print("\n" + "=" * 50)
        print("Additional Info:")
        print(f"Explanation: {result.sql_info.explanation}")
        print(f"Tables used: {', '.join(result.sql_info.tables_used)}")
else:
    print(f"Error: {result.error_message}")

Generated SQL Query:
SELECT MONTH(bill_date) AS month, SUM(total) AS total_spending
FROM bills
WHERE YEAR(bill_date) = 2023
GROUP BY MONTH(bill_date)
ORDER BY MONTH(bill_date);

Additional Info:
Explanation: This query retrieves the total spendings from the 'bills' table for each month in the year 2023, grouping the results by month and ordering them chronologically.
Tables used: bills


In [6]:
result

GenerationResult(success=True, trace_id='bc146827-3cf9-4a32-af58-fbe8dc5e86d1', request_id=1, content='SELECT MONTH(bill_date) AS month, SUM(total) AS total_spending\nFROM bills\nWHERE YEAR(bill_date) = 2023\nGROUP BY MONTH(bill_date)\nORDER BY MONTH(bill_date);', raw_content='{"query":"SELECT MONTH(bill_date) AS month, SUM(total) AS total_spending\\nFROM bills\\nWHERE YEAR(bill_date) = 2023\\nGROUP BY MONTH(bill_date)\\nORDER BY MONTH(bill_date);","explanation":"This query retrieves the total spendings from the \'bills\' table for each month in the year 2023, grouping the results by month and ordering them chronologically.","tables_used":["bills"]}', raw_response=Response(id='resp_0ca2a28ab8b1f79a0068c93706d4b081908a499659609467c2', created_at=1758017286.0, error=None, incomplete_details=None, instructions='Generate SQL query based on the database schema and user request', metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_0ca2a28ab8b

## Example 3: Translator

Translation service now uses structured output for better control and metadata about translations.

In [7]:
# Define schema for translation
class Translation(BaseModel):
    source_text: str = Field(description="Original text")
    source_language: str = Field(description="Detected source language")
    target_language: str = Field(description="Target language")
    translated_text: str = Field(description="Translated text")
    confidence: Optional[float] = Field(default=None, ge=0, le=1, description="Translation confidence")

class TranslatorService(BaseLLMService):
    def __init__(self):
        super().__init__(default_model_name="gpt-4o-mini")
        self.set_rate_limits(max_rpm=120, max_tpm=10_000)
        self.set_concurrency(10)
    
    def translate_to_russian(self, input_paragraph: str, request_id=None):
        """Translate to Russian with structured output"""
        return self._translate_to(input_paragraph, "Russian", request_id)
    
    def translate_to_latin(self, input_paragraph: str, request_id=None):
        """Translate to Latin with structured output"""
        return self._translate_to(input_paragraph, "Latin", request_id)
    
    def _translate_to(self, input_paragraph: str, target_language: str, request_id=None):
        """Generic translation method using structured output"""
        
        generation_request = GenerationRequest(
            user_prompt=f"Translate this text to {target_language}: {input_paragraph}",
            system_prompt=f"Translate the given text to {target_language}",
            response_schema=Translation,  # ← Structured output!
            model="gpt-4o-mini",
            request_id=request_id
        )
        
        result = self.execute_generation(generation_request)
        
        if result.success:
            # Parse and extract just the translation
            data = json.loads(result.content)
            translation = Translation(**data)
            # Set content to just the translated text for compatibility
            result.content = translation.translated_text
            # Store full info
            result.translation_info = translation
        
        return result

# Test Translator - Synchronous
translator = TranslatorService()

texts_to_translate = [
    "Hello, how are you today?",
    "The weather is beautiful.",
    "I love programming with Python."
]

print("=" * 50)
print("Synchronous Translation to Russian:")
print("=" * 50)

for idx, text in enumerate(texts_to_translate):
    result = translator.translate_to_russian(text, request_id=idx)
    if result.success:
        print(f"\nOriginal: {text}")
        print(f"Russian: {result.content}")
        if hasattr(result, 'translation_info'):
            print(f"Detected language: {result.translation_info.source_language}")
    else:
        print(f"Error translating text {idx}: {result.error_message}")

Synchronous Translation to Russian:

Original: Hello, how are you today?
Russian: Привет, как дела сегодня?
Detected language: English

Original: The weather is beautiful.
Russian: Погода прекрасная.
Detected language: English

Original: I love programming with Python.
Russian: Мне нравится программировать на Python.
Detected language: English


## Example 3b: Async Translation

The translator also supports async operations for batch processing. 

**Note**: Running async code in Jupyter notebooks requires special handling. The notebook already has an event loop running, so we need to use `await` directly or use `nest_asyncio` for compatibility.

In [ ]:
# Async translation support
# Note: The async client is now properly configured in ResponsesAPIProvider

class AsyncTranslatorService(TranslatorService):
    async def translate_to_russian_async(self, input_paragraph: str, request_id=None):
        """Async translation to Russian"""
        generation_request = GenerationRequest(
            user_prompt=f"Translate this text to Russian: {input_paragraph}",
            system_prompt="Translate the given text to Russian",
            response_schema=Translation,  # ← Structured output!
            model="gpt-4o-mini",
            request_id=request_id,
            operation_name="translate_to_russian"
        )
        
        result = await self.execute_generation_async(generation_request)
        
        if result.success:
            data = json.loads(result.content)
            translation = Translation(**data)
            result.content = translation.translated_text
        
        return result

# Test async translation
async def translate_batch_async(texts):
    """Translate multiple texts asynchronously"""
    translator = AsyncTranslatorService()
    tasks = []
    
    for idx, text in enumerate(texts):
        task = translator.translate_to_russian_async(text, request_id=idx)
        tasks.append(task)
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    print("=" * 50)
    print("Asynchronous Translation Results:")
    print("=" * 50)
    
    for idx, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"\nError processing text {idx}: {result}")
        elif result.success:
            print(f"\nText {idx}: {texts[idx]}")
            print(f"Translation: {result.content}")
        else:
            print(f"\nError translating text {idx}: {result.error_message}")

# For Jupyter notebooks, use await directly
# The async client fix should resolve the "can't be used in await" error
print("Running async translations...")
await translate_batch_async(texts_to_translate)

## Comparison: Old Pipelines vs New Structured Outputs

This example shows the difference between the old pipeline approach (now removed) and the new structured output approach.

In [ ]:
# Comparison example
print("=" * 60)
print("OLD WAY (Pipelines) - NO LONGER WORKS:")
print("=" * 60)
print("""
# This is what the old examples used (now removed):
pipeline_config = [
    {
        'type': 'SemanticIsolation',
        'params': {'semantic_element_for_extraction': 'just the capital'}
    }
]
generation_request = GenerationRequest(
    formatted_prompt=prompt,
    pipeline_config=pipeline_config  # ❌ REMOVED - TypeError!
)

Problems:
- Complex configuration
- Multiple failure points (String2Dict parsing)
- No type safety
- Fragile and error-prone
""")

print("\n" + "=" * 60)
print("NEW WAY (Structured Outputs) - CLEAN & RELIABLE:")
print("=" * 60)
print("""
# Using structured outputs with Pydantic schema:
class CapitalInfo(BaseModel):
    country: str = Field(description="The country name")
    capital: str = Field(description="The capital city")
    confidence: float = Field(ge=0, le=1)

generation_request = GenerationRequest(
    user_prompt=prompt,
    response_schema=CapitalInfo  # ✅ Structured output!
)

Benefits:
- Simple, clear schema definition
- 100% reliable - no parsing errors
- Type-safe with IDE support
- Rich metadata (confidence, etc.)
""")

## Summary

All three examples from the `examples/` directory have been successfully recreated using structured outputs:

1. **Capital Finder**: Uses `CapitalInfo` schema instead of SemanticIsolation pipeline
2. **SQL Generator**: Uses `SQLQuery` schema for clean SQL extraction  
3. **Translator**: Uses `Translation` schema with full translation metadata

### Key Benefits of the New Approach:
- ✅ **100% Reliable**: No parsing failures
- ✅ **Type-Safe**: Pydantic models with validation
- ✅ **Rich Data**: Get more than just text (confidence, metadata, etc.)
- ✅ **Clean Code**: No complex pipeline configurations
- ✅ **IDE Support**: Autocomplete and type checking

The migration from pipelines to structured outputs makes the code simpler, more reliable, and more maintainable!